In [58]:


#LSTM MOdel Precipitation
import numpy as np
#import mysql.connector
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sqlalchemy import create_engine, text
from Forecast_utils5 import run_precip_forecast_pipeline
from Forecast_utils5 import MilkProductionForecaster

# Function to convert NumPy types to Python native types
def convert_numpy_types(df):
    # Create a copy to avoid modifying the original
    result = df.copy()
    
    # Convert all columns with numpy dtypes to Python native types
    for col in result.columns:
        if pd.api.types.is_integer_dtype(result[col]):
            result[col] = result[col].astype(int)
        elif pd.api.types.is_float_dtype(result[col]):
            result[col] = result[col].astype(float)
    
    return result

# Create SQLAlchemy engine
engine = create_engine(
    'mysql+mysqlconnector://root:Romans17:48@127.0.0.1/livelihoodzones_1'
)
#hha_outliers = HHA_Outliers()

'''conn = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='Romans17:48',
        database='livelihoodzones'
    )

cursor = conn.cursor()
'''

query = """
    SELECT hh_livestock_milk_production_per_species.HhaQuestionnaireSessionId as qid, hha_questionnaire_sessions.CountyId, hha_questionnaire_sessions.LivelihoodZoneId,hha_questionnaire_sessions.WardId,hha_questionnaire_sessions.HouseHoldId, hha_questionnaire_sessions.SubCountyId,
            data_collection_exercise.ExerciseStartDate, Sum(hh_livestock_milk_production_per_species.DailyQntyMilkedInLtrs) as amountmilked,Sum(hh_livestock_milk_production_per_species.DailyQntyConsumedInLtrs) as amountconsumed,Sum(hh_livestock_milk_production_per_species.DailyQntySoldInLtrs) as amountsold, Sum(hh_livestock_milk_production_per_species.PricePerLtr) as PricePerLtr,wards.Shapefile_wardName
    FROM (hh_livestock_milk_production_per_species
          LEFT JOIN hha_questionnaire_sessions ON (hh_livestock_milk_production_per_species.HhaQuestionnaireSessionId = hha_questionnaire_sessions.HhaQuestionnaireSessionId))
          LEFT JOIN data_collection_exercise ON (hha_questionnaire_sessions.DataCollectionExerciseId = data_collection_exercise.DataCollectionExerciseId)LEFT JOIN wards ON (hha_questionnaire_sessions.WardId = wards.WardId)
    WHERE (hha_questionnaire_sessions.CountyId = '46' )
    GROUP BY hh_livestock_milk_production_per_species.HhaQuestionnaireSessionId, hha_questionnaire_sessions.CountyId, hha_questionnaire_sessions.LivelihoodZoneId,hha_questionnaire_sessions.WardId,hha_questionnaire_sessions.HouseHoldId, hha_questionnaire_sessions.SubCountyId,data_collection_exercise.ExerciseStartDate, wards.Shapefile_wardName
"""

#db_df1 = pd.read_sql(query, conn)
db_df1 = pd.read_sql(query, engine)


query = """
    SELECT Seasons.season,Seasons.Season_Index, Seasons.Month,LTAs.Bad_year, LTAs.Good_year
    FROM Seasons LEFT JOIN LTAs ON (Seasons.month = LTAs.month)
   WHERE (LTAs.CountyId = '46')
    """

#Seasons = pd.read_sql(query, conn)
Seasons = pd.read_sql(query, engine)

db_df1['year'] = db_df1['ExerciseStartDate'].dt.year
db_df1['month'] = db_df1['ExerciseStartDate'].dt.strftime('%B') 
db_df1['month_num'] = db_df1['ExerciseStartDate'].dt.month

db_df = db_df1.merge(Seasons, left_on=['month'], right_on=['Month'], how='right')

#conn.close()
#db_df

'''conn = mysql.connector.connect(
        host='127.0.0.1',
        user='root',
        password='Romans17:48',
        database='livelihoodzones'
    )

cursor = conn.cursor()
'''

query = """
    SELECT *
    FROM Precipitation LEFT JOIN counties ON (counties.CountyName = Precipitation.NAME_1)
    WHERE (counties.CountyId = '46')
    """

#precipitation_df = pd.read_sql(query, conn)
precipitation_df = pd.read_sql(query, engine)


prep_df0 = precipitation_df.groupby(['NAME_3','T'])['precipitation'].sum()
prep_df0 = prep_df0.reset_index()

#conn.close()
prep_df0['T'] = pd.to_datetime(prep_df0['T'], errors='coerce')
prep_df0['year'] = prep_df0['T'].dt.year
prep_df0['month_name'] = prep_df0['T'].dt.strftime('%B') 
prep_df0['month_num'] = prep_df0['T'].dt.month
prep_df0
prep_df0 = Seasons.merge(prep_df0, left_on=['Month'], right_on=['month_name'], how='right')
#prep_df0
#Wusi/Kishamba Chala
#unique_wards = prep_df0["NAME_3"].unique()
unique_wards = db_df1[db_df1['Shapefile_wardName'].notna()]
unique_wards = unique_wards['Shapefile_wardName'].unique()
#unique_wards = ['Chala']
for NAME_3 in unique_wards:
    print(f"Processing {NAME_3}...")
    prep_df = prep_df0[prep_df0["NAME_3"] == NAME_3]
    prep_df = prep_df.reset_index()    
    prep_df=prep_df[['season','Season_Index','Month','NAME_3','T','precipitation','year','month_name','month_num']]
    unique_ward = prep_df["NAME_3"].unique()
    #prep_df1=prep_df
    #prep_df['T'] = pd.to_datetime(prep_df['T'])
    #pd.to_datetime(prep_df0['T'], errors='coerce')
    #cutoff_date = pd.to_datetime('2024-0-01')
    #prep_df=prep_df[(prep_df['T'] < cutoff_date)]

    # Call the forecasting function
    results = run_precip_forecast_pipeline(
        prep_df=prep_df,
        seq_length=48,
        forecast_start_year=2016,
        forecast_start_month=1,
        n_future=109
    )

    # Access the results
    model = results["model"]
    scaler = results["scaler"]
    forecast_df = results["forecast_df"]
    test_metrics = results["test_metrics"]

    # Print some results
    print(f"RMSE: {test_metrics.get('rmse')}")
    print(f"Forecast periods with actuals: {results['forecast_metrics'].get('n_with_actuals', 0)}")

    # Access the forecast data
    if not forecast_df.empty:
        print(f"Forecast first 5 periods: \n{forecast_df[['Date', 'Forecasted Precipitation']].head()}")

    print(forecast_df.head())
    #print(metrics)

    #merging Socio-economic data with Rainfall data
    forecast_df1=forecast_df[['Month','Year','Date_Object','Forecasted Precipitation','Forecast Uncertainty (Std Dev)']]
    prep_df1 = forecast_df1.merge(prep_df, left_on=['Month','Year','Date_Object'], right_on=['month_num','year','T'], how='right')
    prep_df2=prep_df1[['year','Forecasted Precipitation','NAME_3','T','precipitation','month_name','month_num']]
    precipitation_forecasts_df=prep_df2
    db_df=db_df[['WardId','HouseHoldId','Shapefile_wardName', 'month', 'year', 'season','Season_Index','amountmilked','Bad_year','Good_year']]
    #db_df=db_df[(db_df['Shapefile_wardName']=="Wusi/Kishamba")&(db_df['year']==2024)]
    #db_df

    #Cleaning data (Outliers)
    def replace_outliers_with_averages(db_df, variables):
        """
        Replace outliers with the average value for each ward and month combination.
        
        Parameters:
        -----------
        db_df : pandas.DataFrame
            DataFrame with outlier detection results
        variables : list
            List of variable names to process
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with outliers replaced by averages
        """
        # Create a clean version with outliers replaced by averages
        db_df_clean = db_df.copy()
        
        # Make sure we have the year_month column (in case it was removed)
        if 'year_month' not in db_df.columns:
            db_df_clean['year_month'] = db_df_clean['year'].astype(str) + '-' + db_df_clean['month'].astype(str)
        
        for variable in variables:
            print(f"\nReplacing outliers for {variable} with ward-month averages")
            outlier_col = f'{variable}_is_outlier'
            
            # Check if outlier column exists
            if outlier_col not in db_df.columns:
                print(f"Warning: No outlier data found for {variable}, skipping replacement")
                continue
            
            # Iterate through each ward and year-month
            for ward in db_df_clean['ward'].unique():
                for year_month in db_df_clean['year_month'].unique():
                    # Get data for this ward and this year-month
                    mask = (db_df_clean['ward'] == ward) & (db_df_clean['year_month'] == year_month)
                    subset = db_df_clean[mask]
                    
                    # Skip if no data
                    if len(subset) == 0:
                        continue
                    
                    # Skip if insufficient data
                    if len(subset) <= 2:
                        print(f"Warning: Ward {ward} in {year_month} has insufficient data for average calculation")
                        continue
                        
                    # Check if there are any outliers in this group
                    if not subset[outlier_col].any():
                        continue
                    
                    # Check if there are any non-outliers to calculate average from
                    if subset[outlier_col].all():
                        print(f"Warning: All values in ward {ward}, {year_month} for {variable} are outliers")
                        # Try to use the ward average instead
                        ward_avg = db_df_clean.loc[(db_df_clean['ward'] == ward) & ~db_df_clean[outlier_col], variable].mean()
                        if np.isnan(ward_avg):
                            print(f"Cannot find suitable replacement for ward {ward}, {year_month}. Keeping original values.")
                            continue
                        replacement_value = ward_avg
                    else:
                        # Calculate mean of non-outlier values
                        non_outlier_mean = subset.loc[~subset[outlier_col], variable].mean()
                        replacement_value = non_outlier_mean
                    
                    # Replace outliers with the mean
                    outlier_mask = mask & db_df_clean[outlier_col]
                    if outlier_mask.any():
                        db_df_clean.loc[outlier_mask, variable] = replacement_value
                        print(f"Replaced {outlier_mask.sum()} outliers in ward {ward}, {year_month} for {variable}")
        
        return db_df_clean

    # Example usage (to be appended to your code):
    # After running analyze_outliers:
    db_df_clean = replace_outliers_with_averages(db_df, variables=['amountmilked'])
    db_df_clean

    unique_ward
    unique_ward1 = unique_ward[0]
    unique_ward2 = unique_ward[0]
    unique_ward1

    db_df_clean1=db_df_clean.groupby(['Shapefile_wardName', 'month', 'year', 'season','Season_Index','Bad_year','Good_year'])[['amountmilked']].mean().reset_index()

    joined_data2 = db_df_clean1.merge(prep_df2, left_on=['Shapefile_wardName', 'year', 'month'], right_on=['NAME_3', 'year', 'month_name'], how='right')

    joined_data3=joined_data2[(joined_data2['Shapefile_wardName']==unique_ward1)&(joined_data2['year']>2016)]

    data_numeric = joined_data3.assign(**{col: joined_data3[col].map(lambda x: x.toordinal()) 
                                        for col in joined_data3.select_dtypes(include=['datetime64'])})
    

    data_numeric = data_numeric.sort_values(by="T")
    print(data_numeric.head())

    # Define features (make sure to include Shapefile_wardName if you need it for other purposes)
    features = ["year", "month_num", "Season_Index", "precipitation", 
                "Forecasted Precipitation", "amountmilked", "months_gap"]


    # Call the function with error handling
    if data_numeric is None or data_numeric.empty:
        print("Warning: data_numeric is empty or None. Cannot proceed with forecasting.")
        # Create empty results structure
        results = {
            'model': None,
            'scaler': None,
            'training_history': None,
            'evaluation_metrics': {},
            'test_results': pd.DataFrame(),
            'forecast_results': pd.DataFrame(),
            'feature_indices': {},
            'data_month_to_season': {}
        }
        forecast_df = results["forecast_results"]
        test_results = results["test_results"]
        evaluation_metrics = results["evaluation_metrics"]

    else:
        # Call the function
        results = MilkProductionForecaster(
            data_numeric.copy(),
            features=features,
            n_future=16,
            external_precip_forecasts=precipitation_forecasts_df
        )

        # Access the ward name that was used

        forecast_df = results["forecast_results"]
        test_results = results["test_results"]
        evaluation_metrics = results["evaluation_metrics"]
        #print(f"Analysis done for ward: {results['ward_name']}")

        # The DataFrames will automatically include the correct ward name
        print(results['forecast_results'][['Ward', 'Month', 'Year', 'Forecasted Amount Milked']].head())
        forecast_df.rename(columns={'Months Gap': 'Months_Gap','Forecasted Amount Milked': 'Forecasted_Value','Actual (if available)': 'Actual','Forecast Uncertainty (Std Dev)': 'Forecast_Uncertainty','Lower Bound (95%)': 'Lower_Bound','Upper Bound (95%)': 'Upper_Bound','Percent Error': 'Percent_Error'}, inplace=True)
        forecast_df5=forecast_df[['Month','Year','Season_Index','Precipitation','Months_Gap','Date','Date_Object','Forecasted_Value','Actual','Forecast_Uncertainty','Lower_Bound','Upper_Bound','Error','Percent_Error','Last_Actual_Value','Month1_Forecast','Month2_Forecast','Month3_Forecast','Evaluation_Metrics']]
        forecast_df5['Ward']=unique_ward2
        forecast_df5['Indicator']="TotalDailyQntyMilkedInLtrs"
        forecast_df5=forecast_df5[['Ward','Month','Year','Season_Index','Precipitation','Months_Gap','Date','Date_Object','Indicator','Forecasted_Value','Actual','Forecast_Uncertainty','Lower_Bound','Upper_Bound','Error','Percent_Error','Last_Actual_Value','Month1_Forecast','Month2_Forecast','Month3_Forecast','Evaluation_Metrics']]
        forecast_df5

        # Process forecast_df5 to make it database-ready by converting all columns to Python native types
        for col in forecast_df5.columns:
            # For numeric columns
            if pd.api.types.is_numeric_dtype(forecast_df5[col]):
                forecast_df5[col] = forecast_df5[col].apply(
                    lambda x: float(x) if not pd.isna(x) else None
                )
            # For timestamp columns
            elif pd.api.types.is_datetime64_dtype(forecast_df5[col]):
                forecast_df5[col] = forecast_df5[col].apply(
                    lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if not pd.isna(x) else None
                )
            # For string columns
            else:
                forecast_df5[col] = forecast_df5[col].apply(
                    lambda x: str(x) if not pd.isna(x) else None
                )

        # Handle Date_Object column
        if 'Date_Object' in forecast_df5.columns:
            forecast_df5['Date_Object'] = forecast_df5['Date_Object'].apply(
                lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if hasattr(x, 'strftime') and x is not None 
                        else x if isinstance(x, str) else None
            )

        # STEP 1: Get all Ward/Month/Year combinations where Last_Actual_Value is not null in the database
        existing_special_rows = []
        try:
            query = text("""
                SELECT Ward, Month, Year FROM Predictions 
                WHERE Last_Actual_Value IS NOT NULL
            """)
            with engine.connect() as conn:
                result = conn.execute(query)
                existing_special_rows = [(row[0], row[1], row[2]) for row in result]
            print(f"Found {len(existing_special_rows)} existing special rows in database")
        except Exception as e:
            print(f"Error checking for existing special rows: {e}")

        # STEP 2: Identify special rows in forecast_df5
        special_rows_df = forecast_df5[forecast_df5['Last_Actual_Value'].notnull()].copy()
        special_rows_keys = [(row['Ward'], row['Month'], row['Year']) for _, row in special_rows_df.iterrows()]
        print(f"Found {len(special_rows_keys)} special rows in forecast_df5")

        # STEP 3: Handle each row in forecast_df5
        insert_count = 0
        skip_count = 0

        for idx, row in forecast_df5.iterrows():
            ward = row['Ward']
            month = row['Month']
            year = row['Year']
            row_key = (ward, month, year)
            is_special = row['Last_Actual_Value'] is not None
            
            # Case 1: This is a special row that's not in the database yet - insert it
            if is_special and row_key not in existing_special_rows:
                # First delete any non-special rows with same key
                try:
                    delete_query = text("""
                        DELETE FROM Predictions 
                        WHERE Ward = :ward AND Month = :month AND Year = :year
                    """)
                    with engine.begin() as conn:
                        conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
                    print(f"Cleared existing rows for new special row {row_key}")
                except Exception as e:
                    print(f"Error clearing rows for new special row {row_key}: {e}")
                    
                # Now insert
                should_insert = True
            
            # Case 2: This is a non-special row, and no special row exists in DB - insert it
            elif not is_special and row_key not in existing_special_rows:
                # First delete any existing rows with same key (which must be non-special)
                try:
                    delete_query = text("""
                        DELETE FROM Predictions 
                        WHERE Ward = :ward AND Month = :month AND Year = :year
                    """)
                    with engine.begin() as conn:
                        conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
                except Exception as e:
                    print(f"Error clearing rows for non-special row {row_key}: {e}")
                    
                # Now insert
                should_insert = True
            
            # Case 3: This is a special row that already exists in DB - update it
            elif is_special and row_key in existing_special_rows:
                # This case means we're updating a special row in the DB
                try:
                    delete_query = text("""
                        DELETE FROM Predictions 
                        WHERE Ward = :ward AND Month = :month AND Year = :year
                    """)
                    with engine.begin() as conn:
                        conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
                    print(f"Deleted existing special row {row_key} for update")
                except Exception as e:
                    print(f"Error deleting existing special row {row_key} for update: {e}")
                    
                # Now insert the updated row
                should_insert = True
                
            # Case 4: This is a non-special row, but a special row exists in DB - skip it
            else:  # not is_special and row_key in existing_special_rows
                print(f"Skipping non-special row {row_key} because special row exists in DB")
                skip_count += 1
                should_insert = False
            
            # Insert the row if needed
            if should_insert:
                params = {}
                for col in forecast_df5.columns:
                    value = row[col]
                    
                    # Convert numpy and pandas types to Python native types
                    if hasattr(value, 'dtype'):
                        if np.issubdtype(value.dtype, np.integer):
                            params[col.lower()] = int(value)
                        elif np.issubdtype(value.dtype, np.floating):
                            params[col.lower()] = float(value) if not np.isnan(value) else None
                        else:
                            params[col.lower()] = str(value) if value is not None else None
                    elif pd.isna(value):
                        params[col.lower()] = None
                    else:
                        params[col.lower()] = value
                
                # Add default value for Indicator if not in the dataframe
                if 'indicator' not in params:
                    params['indicator'] = 'amountmilked'  # Default value
                    
                # Add default value for Forecasted_Value if not in the dataframe
                if 'forecasted_value' not in params and 'forecasted_amount_milked' in params:
                    params['forecasted_value'] = params['forecasted_amount_milked']
                
                # Get database column names to ensure we're only inserting valid columns
                try:
                    inspect_query = text("""
                        SHOW COLUMNS FROM Predictions
                    """)
                    with engine.connect() as conn:
                        result = conn.execute(inspect_query)
                        db_columns = [row[0].lower() for row in result]
                        
                    # Build the insert query dynamically based on available columns
                    columns = []
                    values = []
                    insert_params = {}
                    
                    for key, value in params.items():
                        if key.lower() in db_columns:
                            columns.append(key.lower())
                            values.append(f":{key.lower()}")
                            insert_params[key.lower()] = value
                    
                    # Create the INSERT statement
                    columns_str = ", ".join(columns)
                    values_str = ", ".join(values)
                    insert_query = text(f"""
                        INSERT INTO Predictions ({columns_str}) 
                        VALUES ({values_str})
                    """)
                    
                    with engine.begin() as conn:
                        conn.execute(insert_query, insert_params)
                    
                    insert_count += 1
                    if is_special:
                        print(f"Inserted/updated special row {row_key}")
                    elif insert_count % 10 == 0:
                        print(f"Inserted {insert_count} rows so far")
                        
                except Exception as e:
                    print(f"Error inserting record {row_key}: {e}")
                    print("Parameters used:")
                    for key, value in params.items():
                        print(f"  {key}: {value} (type: {type(value)})")

        print(f"Inserted/updated {insert_count} rows, skipped {skip_count} rows")
        print("Process completed successfully.")

Processing Wumingu/Kishushe...


/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo

RMSE: 1303.1643616938636
Forecast periods with actuals: 106
Forecast first 5 periods: 
      Date  Forecasted Precipitation
0  2015-12                384.901354
1  2016-01               -992.842439
2  2016-02               1412.400138
3  2016-03               3586.550367
4  2016-04               2282.743470
   Month  Year  Season_Index  Months_Gap     Date Date_Object  \
0     12  2015           4.0           1  2015-12  2015-12-01   
1      1  2016           1.0           1  2016-01  2016-01-01   
2      2  2016           1.0           1  2016-02  2016-02-01   
3      3  2016           2.0           1  2016-03  2016-03-01   
4      4  2016           2.0           1  2016-04  2016-04-01   

   Forecasted Precipitation  Actual_if_available  \
0                384.901354          3160.432106   
1               -992.842439          1245.101002   
2               1412.400138           353.322944   
3               3586.550367          1105.499390   
4               2282.743470          417

/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have 

                 Ward  Month  Year  Forecasted Amount Milked
0  Shapefile_wardName      9  2023                  6.304548
1  Shapefile_wardName     10  2023                  6.306426
2  Shapefile_wardName     11  2023                  6.503785
3  Shapefile_wardName     12  2023                  6.702741
4  Shapefile_wardName      1  2024                  7.049865
Found 5 existing special rows in database
Found 1 special rows in forecast_df5
Inserted 10 rows so far
Deleted existing special row ('Wumingu/Kishushe', 9.0, 2024.0) for update
Inserted/updated special row ('Wumingu/Kishushe', 9.0, 2024.0)
Inserted/updated 16 rows, skipped 0 rows
Process completed successfully.
Processing Kasigau...


...

                 Ward  Month  Year  Forecasted Amount Milked
0  Shapefile_wardName      9  2023                  1.245994
1  Shapefile_wardName     10  2023                  1.503109
2  Shapefile_wardName     11  2023                  1.457160
3  Shapefile_wardName     12  2023                  1.899018
4  Shapefile_wardName      1  2024                  2.178536
Found 5 existing special rows in database
Found 1 special rows in forecast_df5
Inserted 10 rows so far
Cleared existing rows for new special row ('Mboghoni', 9.0, 2024.0)
Inserted/updated special row ('Mboghoni', 9.0, 2024.0)
Inserted/updated 16 rows, skipped 0 rows
Process completed successfully.
Processing Chala...


/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo

RMSE: 2865.50734737641
Forecast periods with actuals: 106
Forecast first 5 periods: 
      Date  Forecasted Precipitation
0  2015-12               1550.171117
1  2016-01              -2302.531296
2  2016-02                659.521086
3  2016-03               9066.836894
4  2016-04               6720.195981
   Month  Year  Season_Index  Months_Gap     Date Date_Object  \
0     12  2015           4.0           1  2015-12  2015-12-01   
1      1  2016           1.0           1  2016-01  2016-01-01   
2      2  2016           1.0           1  2016-02  2016-02-01   
3      3  2016           2.0           1  2016-03  2016-03-01   
4      4  2016           2.0           1  2016-04  2016-04-01   

   Forecasted Precipitation  Actual_if_available  \
0               1550.171117          6433.005417   
1              -2302.531296          3730.123306   
2                659.521086           882.925085   
3               9066.836894          2904.682688   
4               6720.195981         10624.

/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/home/ebenezer/miniconda3/envs/geo2_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have 

                 Ward  Month  Year  Forecasted Amount Milked
0  Shapefile_wardName      9  2023                  1.437514
1  Shapefile_wardName     10  2023                  1.446101
2  Shapefile_wardName     11  2023                  1.519245
3  Shapefile_wardName     12  2023                  1.530458
4  Shapefile_wardName      1  2024                  1.656888
Found 6 existing special rows in database
Found 1 special rows in forecast_df5
Inserted 10 rows so far
Deleted existing special row ('Chala', 9.0, 2024.0) for update
Inserted/updated special row ('Chala', 9.0, 2024.0)
Inserted/updated 16 rows, skipped 0 rows
Process completed successfully.


In [30]:
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd

# Create engine
engine = create_engine('mysql+mysqlconnector://root:Romans17:48@127.0.0.1/livelihoodzones_1')

# Process forecast_df5 to make it database-ready by converting all columns to Python native types
for col in forecast_df5.columns:
    # For numeric columns
    if pd.api.types.is_numeric_dtype(forecast_df5[col]):
        forecast_df5[col] = forecast_df5[col].apply(
            lambda x: float(x) if not pd.isna(x) else None
        )
    # For timestamp columns
    elif pd.api.types.is_datetime64_dtype(forecast_df5[col]):
        forecast_df5[col] = forecast_df5[col].apply(
            lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if not pd.isna(x) else None
        )
    # For string columns
    else:
        forecast_df5[col] = forecast_df5[col].apply(
            lambda x: str(x) if not pd.isna(x) else None
        )

# Handle Date_Object column
if 'Date_Object' in forecast_df5.columns:
    forecast_df5['Date_Object'] = forecast_df5['Date_Object'].apply(
        lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if hasattr(x, 'strftime') and x is not None 
                else x if isinstance(x, str) else None
    )

# STEP 1: Get all Ward/Month/Year combinations where Last_Actual_Value is not null in the database
existing_special_rows = []
try:
    query = text("""
        SELECT Ward, Month, Year FROM Predictions 
        WHERE Last_Actual_Value IS NOT NULL
    """)
    with engine.connect() as conn:
        result = conn.execute(query)
        existing_special_rows = [(row[0], row[1], row[2]) for row in result]
    print(f"Found {len(existing_special_rows)} existing special rows in database")
except Exception as e:
    print(f"Error checking for existing special rows: {e}")

# STEP 2: Identify special rows in forecast_df5
special_rows_df = forecast_df5[forecast_df5['Last_Actual_Value'].notnull()].copy()
special_rows_keys = [(row['Ward'], row['Month'], row['Year']) for _, row in special_rows_df.iterrows()]
print(f"Found {len(special_rows_keys)} special rows in forecast_df5")

# STEP 3: Handle each row in forecast_df5
insert_count = 0
skip_count = 0

for idx, row in forecast_df5.iterrows():
    ward = row['Ward']
    month = row['Month']
    year = row['Year']
    row_key = (ward, month, year)
    is_special = row['Last_Actual_Value'] is not None
    
    # Case 1: This is a special row that's not in the database yet - insert it
    if is_special and row_key not in existing_special_rows:
        # First delete any non-special rows with same key
        try:
            delete_query = text("""
                DELETE FROM Predictions 
                WHERE Ward = :ward AND Month = :month AND Year = :year
            """)
            with engine.begin() as conn:
                conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
            print(f"Cleared existing rows for new special row {row_key}")
        except Exception as e:
            print(f"Error clearing rows for new special row {row_key}: {e}")
            
        # Now insert
        should_insert = True
    
    # Case 2: This is a non-special row, and no special row exists in DB - insert it
    elif not is_special and row_key not in existing_special_rows:
        # First delete any existing rows with same key (which must be non-special)
        try:
            delete_query = text("""
                DELETE FROM Predictions 
                WHERE Ward = :ward AND Month = :month AND Year = :year
            """)
            with engine.begin() as conn:
                conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
        except Exception as e:
            print(f"Error clearing rows for non-special row {row_key}: {e}")
            
        # Now insert
        should_insert = True
    
    # Case 3: This is a special row that already exists in DB - update it
    elif is_special and row_key in existing_special_rows:
        # This case means we're updating a special row in the DB
        try:
            delete_query = text("""
                DELETE FROM Predictions 
                WHERE Ward = :ward AND Month = :month AND Year = :year
            """)
            with engine.begin() as conn:
                conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
            print(f"Deleted existing special row {row_key} for update")
        except Exception as e:
            print(f"Error deleting existing special row {row_key} for update: {e}")
            
        # Now insert the updated row
        should_insert = True
        
    # Case 4: This is a non-special row, but a special row exists in DB - skip it
    else:  # not is_special and row_key in existing_special_rows
        print(f"Skipping non-special row {row_key} because special row exists in DB")
        skip_count += 1
        should_insert = False
    
    # Insert the row if needed
    if should_insert:
        params = {}
        for col in forecast_df5.columns:
            value = row[col]
            
            # Convert numpy and pandas types to Python native types
            if hasattr(value, 'dtype'):
                if np.issubdtype(value.dtype, np.integer):
                    params[col.lower()] = int(value)
                elif np.issubdtype(value.dtype, np.floating):
                    params[col.lower()] = float(value) if not np.isnan(value) else None
                else:
                    params[col.lower()] = str(value) if value is not None else None
            elif pd.isna(value):
                params[col.lower()] = None
            else:
                params[col.lower()] = value
        
        insert_query = text("""
            INSERT INTO Predictions (
                Ward, Month, Year, Season_Index, Precipitation, 
                Months_Gap, Date, Date_Object,Indicator, Forecasted_Value, 
                Actual, Forecast_Uncertainty, Lower_Bound, Upper_Bound, 
                Error, Percent_Error, Last_Actual_Value, Month1_Forecast, 
                Month2_Forecast, Month3_Forecast, Evaluation_Metrics
            ) 
            VALUES (
                :ward, :month, :year, :season_index, :precipitation, 
                :months_gap, :date, :date_object,:Indicator, :Forecasted_Value, 
                :actual, :forecast_uncertainty, :lower_bound, :upper_bound, 
                :error, :percent_error, :last_actual_value, :month1_forecast, 
                :month2_forecast, :month3_forecast, :evaluation_metrics
            )
        """)
        
        try:
            with engine.begin() as conn:
                conn.execute(insert_query, params)
            
            insert_count += 1
            if is_special:
                print(f"Inserted/updated special row {row_key}")
            elif insert_count % 10 == 0:
                print(f"Inserted {insert_count} rows so far")
                
        except Exception as e:
            print(f"Error inserting record {row_key}: {e}")

print(f"Inserted/updated {insert_count} rows, skipped {skip_count} rows")
print("Process completed successfully.")


Found 0 existing special rows in database
Found 1 special rows in forecast_df5
Error inserting record ('Chala', 9.0, 2023.0): (sqlalchemy.exc.InvalidRequestError) A value is required for bind parameter 'Indicator'
[SQL: 
            INSERT INTO Predictions (
                Ward, Month, Year, Season_Index, Precipitation, 
                Months_Gap, Date, Date_Object,Indicator, Forecasted_Value, 
                Actual, Forecast_Uncertainty, Lower_Bound, Upper_Bound, 
                Error, Percent_Error, Last_Actual_Value, Month1_Forecast, 
                Month2_Forecast, Month3_Forecast, Evaluation_Metrics
            ) 
            VALUES (
                %(ward)s, %(month)s, %(year)s, %(season_index)s, %(precipitation)s, 
                %(months_gap)s, %(date)s, %(date_object)s,%(Indicator)s, %(Forecasted_Value)s, 
                %(actual)s, %(forecast_uncertainty)s, %(lower_bound)s, %(upper_bound)s, 
                %(error)s, %(percent_error)s, %(last_actual_value)s, %(month1

In [31]:
forecast_df5

,Ward,Month,Year,Season_Index,Precipitation,Months_Gap,Date,Date_Object,Indicator,Forecasted_Value,...,Forecast_Uncertainty,Lower_Bound,Upper_Bound,Error,Percent_Error,Last_Actual_Value,Month1_Forecast,Month2_Forecast,Month3_Forecast,Evaluation_Metrics
0,Chala,9.0,2023.0,3.0,608.268097,1.0,2023-09,2023-09-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.331252,...,0.300635,0.742007,1.920497,-0.331252,-33.125212,None,None,None,None,None
1,Chala,10.0,2023.0,4.0,3068.578591,1.0,2023-10,2023-10-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.264419,...,0.304776,0.667058,1.861779,-0.514419,-68.589158,None,None,None,None,None
2,Chala,11.0,2023.0,4.0,7351.409966,1.0,2023-11,2023-11-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.290074,...,0.305730,0.690843,1.889305,NaN,NaN,None,None,None,None,None
3,Chala,12.0,2023.0,4.0,7914.398254,1.0,2023-12,2023-12-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.322360,...,0.307637,0.719390,1.925329,NaN,NaN,None,None,None,None,None
4,Chala,1.0,2024.0,1.0,2307.736630,1.0,2024-01,2024-01-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.431846,...,0.327877,0.789206,2.074486,1.901488,57.044625,None,None,None,None,None
5,Chala,2.0,2024.0,1.0,1649.708920,1.0,2024-02,2024-02-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.415849,...,0.312404,0.803536,2.028161,-0.165849,-13.267881,None,None,None,None,None
6,Chala,3.0,2024.0,2.0,6154.100547,1.0,2024-03,2024-03-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.339104,...,0.311615,0.728338,1.949870,0.160896,10.726380,None,None,None,None,None
7,Chala,4.0,2024.0,2.0,8295.825242,1.0,2024-04,2024-04-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.374597,...,0.309644,0.767693,1.981500,0.292070,17.524210,None,None,None,None,None
8,Chala,5.0,2024.0,2.0,2351.137961,1.0,2024-05,2024-05-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.379143,...,0.314264,0.763186,1.995101,0.454190,24.773997,None,None,None,None,None
9,Chala,6.0,2024.0,3.0,435.267969,1.0,2024-06,2024-06-01 00:00:00,TotalDailyQntyMilkedInLtrs,1.366939,...,0.317654,0.744337,1.989541,NaN,NaN,None,None,None,None,None


In [36]:
from sqlalchemy import create_engine, text
import numpy as np
import pandas as pd

# Create engine
engine = create_engine('mysql+mysqlconnector://root:Romans17:48@127.0.0.1/livelihoodzones_1')

# Process forecast_df5 to make it database-ready by converting all columns to Python native types
for col in forecast_df5.columns:
    # For numeric columns
    if pd.api.types.is_numeric_dtype(forecast_df5[col]):
        forecast_df5[col] = forecast_df5[col].apply(
            lambda x: float(x) if not pd.isna(x) else None
        )
    # For timestamp columns
    elif pd.api.types.is_datetime64_dtype(forecast_df5[col]):
        forecast_df5[col] = forecast_df5[col].apply(
            lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if not pd.isna(x) else None
        )
    # For string columns
    else:
        forecast_df5[col] = forecast_df5[col].apply(
            lambda x: str(x) if not pd.isna(x) else None
        )

# Handle Date_Object column
if 'Date_Object' in forecast_df5.columns:
    forecast_df5['Date_Object'] = forecast_df5['Date_Object'].apply(
        lambda x: x.strftime('%Y-%m-%d %H:%M:%S') if hasattr(x, 'strftime') and x is not None 
                else x if isinstance(x, str) else None
    )

# STEP 1: Get all Ward/Month/Year combinations where Last_Actual_Value is not null in the database
existing_special_rows = []
try:
    query = text("""
        SELECT Ward, Month, Year FROM Predictions 
        WHERE Last_Actual_Value IS NOT NULL
    """)
    with engine.connect() as conn:
        result = conn.execute(query)
        existing_special_rows = [(row[0], row[1], row[2]) for row in result]
    print(f"Found {len(existing_special_rows)} existing special rows in database")
except Exception as e:
    print(f"Error checking for existing special rows: {e}")

# STEP 2: Identify special rows in forecast_df5
special_rows_df = forecast_df5[forecast_df5['Last_Actual_Value'].notnull()].copy()
special_rows_keys = [(row['Ward'], row['Month'], row['Year']) for _, row in special_rows_df.iterrows()]
print(f"Found {len(special_rows_keys)} special rows in forecast_df5")

# STEP 3: Handle each row in forecast_df5
insert_count = 0
skip_count = 0

for idx, row in forecast_df5.iterrows():
    ward = row['Ward']
    month = row['Month']
    year = row['Year']
    row_key = (ward, month, year)
    is_special = row['Last_Actual_Value'] is not None
    
    # Case 1: This is a special row that's not in the database yet - insert it
    if is_special and row_key not in existing_special_rows:
        # First delete any non-special rows with same key
        try:
            delete_query = text("""
                DELETE FROM Predictions 
                WHERE Ward = :ward AND Month = :month AND Year = :year
            """)
            with engine.begin() as conn:
                conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
            print(f"Cleared existing rows for new special row {row_key}")
        except Exception as e:
            print(f"Error clearing rows for new special row {row_key}: {e}")
            
        # Now insert
        should_insert = True
    
    # Case 2: This is a non-special row, and no special row exists in DB - insert it
    elif not is_special and row_key not in existing_special_rows:
        # First delete any existing rows with same key (which must be non-special)
        try:
            delete_query = text("""
                DELETE FROM Predictions 
                WHERE Ward = :ward AND Month = :month AND Year = :year
            """)
            with engine.begin() as conn:
                conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
        except Exception as e:
            print(f"Error clearing rows for non-special row {row_key}: {e}")
            
        # Now insert
        should_insert = True
    
    # Case 3: This is a special row that already exists in DB - update it
    elif is_special and row_key in existing_special_rows:
        # This case means we're updating a special row in the DB
        try:
            delete_query = text("""
                DELETE FROM Predictions 
                WHERE Ward = :ward AND Month = :month AND Year = :year
            """)
            with engine.begin() as conn:
                conn.execute(delete_query, {"ward": ward, "month": month, "year": year})
            print(f"Deleted existing special row {row_key} for update")
        except Exception as e:
            print(f"Error deleting existing special row {row_key} for update: {e}")
            
        # Now insert the updated row
        should_insert = True
        
    # Case 4: This is a non-special row, but a special row exists in DB - skip it
    else:  # not is_special and row_key in existing_special_rows
        print(f"Skipping non-special row {row_key} because special row exists in DB")
        skip_count += 1
        should_insert = False
    
    # Insert the row if needed
    if should_insert:
        params = {}
        for col in forecast_df5.columns:
            value = row[col]
            
            # Convert numpy and pandas types to Python native types
            if hasattr(value, 'dtype'):
                if np.issubdtype(value.dtype, np.integer):
                    params[col.lower()] = int(value)
                elif np.issubdtype(value.dtype, np.floating):
                    params[col.lower()] = float(value) if not np.isnan(value) else None
                else:
                    params[col.lower()] = str(value) if value is not None else None
            elif pd.isna(value):
                params[col.lower()] = None
            else:
                params[col.lower()] = value
        
        # Add default value for Indicator if not in the dataframe
        if 'indicator' not in params:
            params['indicator'] = 'amountmilked'  # Default value
            
        # Add default value for Forecasted_Value if not in the dataframe
        if 'forecasted_value' not in params and 'forecasted_amount_milked' in params:
            params['forecasted_value'] = params['forecasted_amount_milked']
        
        # Get database column names to ensure we're only inserting valid columns
        try:
            inspect_query = text("""
                SHOW COLUMNS FROM Predictions
            """)
            with engine.connect() as conn:
                result = conn.execute(inspect_query)
                db_columns = [row[0].lower() for row in result]
                
            # Build the insert query dynamically based on available columns
            columns = []
            values = []
            insert_params = {}
            
            for key, value in params.items():
                if key.lower() in db_columns:
                    columns.append(key.lower())
                    values.append(f":{key.lower()}")
                    insert_params[key.lower()] = value
            
            # Create the INSERT statement
            columns_str = ", ".join(columns)
            values_str = ", ".join(values)
            insert_query = text(f"""
                INSERT INTO Predictions ({columns_str}) 
                VALUES ({values_str})
            """)
            
            with engine.begin() as conn:
                conn.execute(insert_query, insert_params)
            
            insert_count += 1
            if is_special:
                print(f"Inserted/updated special row {row_key}")
            elif insert_count % 10 == 0:
                print(f"Inserted {insert_count} rows so far")
                
        except Exception as e:
            print(f"Error inserting record {row_key}: {e}")
            print("Parameters used:")
            for key, value in params.items():
                print(f"  {key}: {value} (type: {type(value)})")

print(f"Inserted/updated {insert_count} rows, skipped {skip_count} rows")
print("Process completed successfully.")

Found 1 existing special rows in database
Found 1 special rows in forecast_df5
Inserted 10 rows so far
Skipping non-special row ('Chala', 2.0, 2024.0) because special row exists in DB
Cleared existing rows for new special row ('Chala', 3.0, 2024.0)
Inserted/updated special row ('Chala', 3.0, 2024.0)
Inserted/updated 15 rows, skipped 1 rows
Process completed successfully.


In [40]:
prep_df

,season,Season_Index,Month,NAME_3,T,precipitation,year,month_name,month_num
0,Lean_Season_LR,3,September,Chawia,2024-09-01,780.277454,2024,September,9


In [41]:
unique_wards

array(['Bura', 'Chala', 'Chawia', 'Kaloleni', 'Kasigau', 'Mahoo',
       'Marungu', 'Mata', 'Mboghoni', 'Mbololo', 'Mwanda/Mgange',
       'Mwatate', 'Ngolia', "Rong'E", 'Sagalla', 'Wumingu/Kishushe',
       'Wundanyi/Mbale', 'Wusi/Kishamba'], dtype=object)

In [ ]:
#unique_wards1 = db_df1[db_df1['Shapefile_wardName'=="None"]]
unique_wards1 = db_df1[db_df1['Shapefile_wardName'].notna()]
unique_wards1 = unique_wards1['Shapefile_wardName'].unique()
#unique_wards1=db_df1[(db_df1['Shapefile_wardName']!="None")]
unique_wards1 = unique_wards1['Shapefile_wardName'].unique()

In [57]:
unique_wards1

array(['Wumingu/Kishushe', 'Kasigau', 'Wusi/Kishamba', 'Ngolia',
       'Mboghoni', 'Chala'], dtype=object)